## GPyTorch: Gaussian Process Modeling  At HPC Scales

### Authors/Presenters: Tiffany Christian (Northwestern), Yuyang (Edward) Tiang (University of Chicago), Marieme Ngom (ANL), Carlo Graziani (ANL) 
### Other Science Authorship Credit: Julie Bessac (ANL/Virginia Tech), Vishwas Rao (ANL), Brandi Gamelin (ANL)

The target audience of this tutorial is scientists who are interested in the possibility of carrying out Gaussian Process (GP) modeling at very large scales. 

This possibility is novel and exciting, because while GPs have always been attractive as a data modeling choice for their conceptual clarity, great flexibility, and mathematical rigor, they have has suffered from severe scalability limitations. With $N$ training samples, a "classical" GP treatment yields a storage cost that scales as $\mathcal{O}(N^2)$, and a time to solution that scales as $\mathcal{O}(N^3)$. These scalings have kept GP modeling from achieving the kind of success at large data scales that deep learning methods have attained.  

This situation has changed recently in consequence of two developments. One is the the rise of fast and accurate approximation methods that scale essentially as $\mathcal{O}(N^0)$ --- constant time! The other is the transparent implementation of such methods in a library --- $\texttt{GPyTorch}$ --- that sits on top of $\texttt{PyTorch}$. The reason this latter factor is significant is 
* $\texttt{PyTorch}$ provides built-in GPU acceleration to any library that is built on top of it, if GPU hardware is locally available; and 
* $\texttt{PyTorch}$ is one of the first libraries deployed to any scientific software stack on a new HPC platform  
As a consequence, $\texttt{GPyTorch}$ not only benefits from GPU acceleration effortlessly where a GPU is available, but it also gets a free ride onto any new HPC platform with a hetereogeneous architecture --- which is to say, any new HPC platform.  

The $\texttt{GPyTorch}$ documentation website is at [https://docs.gpytorch.ai/en/stable/index.html](https://docs.gpytorch.ai/en/stable/index.html), and the main website, including developer credits, is at [https://gpytorch.ai/](https://gpytorch.ai/). The doc websites contains several introductory tutorials, and an extensive API documentation system. The main website links to the github development repo, which among other things hosts an active discussion/questions site.

In this tutorial, we will assume some basic knowledge of GP theory, and introduce elements of the $\texttt{GPyTorch}$ API as required, commenting on their purpose.  The objective is to highlight scale, and specifically HPC scale.

We will use data from a climate science application which was kindly shared with us by Julie Bessac (ANL/Virginia Tech), Vishwas Rao (ANL), Brandi Gamelin (ANL), and Tiffany Christian (Northwestern). The data comes from a climate simulation, which output a measure of drought called Standard Vapor Pressure Deficit Index (SVDI) at 12 km resolution and 1 day cadence over the continental United States (CONUS). The data has been collected into contiguous "tiles" of 16x16 points.

We will start with a simple time-series regression, to familiarize ourselves with the $\texttt{GPyTorch}$ interface. First, a few obligatory imports.

In [ ]:
# We need PyTorch and GPyTorch
import torch
from torch import Tensor
import gpytorch
# Gaussian processes really don't work well without 64-bit precision
torch.set_default_dtype(torch.float64)

# And a few more players
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
# Read in the data
input_data_all = np.load("/lus/eagle/projects/datascience/cgraziani/Drought_data/droughtSVDI_data_input.npy")
output_data_all = np.load("/lus/eagle/projects/datascience/cgraziani/Drought_data/droughtSVDI_data_ouput.npy")

print (input_data_all.shape, output_data_all.shape)
# 365 days (one year), 18 N-S * 39 W-E tiles, each 16*16 points.
# The input data codes latitude and longitude.
# The output data codes SVDI

#### Simple time-series prediction
We choose a tile from the data. In this case it's the 12th N-S tile and 30th W-E tile (no particular reason).  We chose the point (8,8) within that tile to extract a time-series of SVDI. The time series begins at day 10. We will train for 120 days of data. Then we will extend a window 30 days past the training window to see how well we can forecast SVDI.  

First, prepare the data.

In [ ]:
# Easiest case: 1-D regression-prediction
long=12 ; lat=30 ; point = (8,8)
Start_Day=10 ; N_Train = 120

# note: input locations are vectors even if they are 1-dimensional
train_x = np.arange(Start_Day, Start_Day+N_Train, dtype=np.float64).reshape((N_Train,1)) 
train_x = Tensor(train_x)

train_y=output_data_all[Start_Day:Start_Day+N_Train, long, lat, point[0], point[1]]
train_y = np.array(train_y)
train_y = Tensor(train_y)


Now prepare the gpytorch model. There is a detailed explanation of the structure of this model at [the gpytorch documentation page](https://docs.gpytorch.ai/en/stable/index.html) and an annotated worked example at their [regression tutorial](https://docs.gpytorch.ai/en/stable/examples/01_Exact_GPs/Simple_GP_Regression.html). This example is very similar to that one. It differs in the choice of GP kernel --- here we choose the Matern kernel, instead of the squared-exponential (AKA "RBF") kernel.

In [ ]:
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        # A GP is defined by a mean function and a covariance function,
        #
        # Here is the mean
        self.mean_module = gpytorch.means.ConstantMean()
        # And here is the covariance
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu = 2.5))
        
 
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# The likelihood provides the model for any noise to be added to the model
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# The model instance.
model = GPRegressionModel(train_x, train_y, likelihood)


Time to set up for training. We need an optimizer. Surprisingly, Adam tends to work well, even though the parameter spaces are not very high-dimensional, $d=4$, vs. the $d=10^6$ tunable weight parameter spaces common in neural networks. We print out parameter values as we train. The way to discover the attributes associated with these is documented [here](https://docs.gpytorch.ai/en/stable/examples/00_Basic_Usage/Hyperparameters.html).  

In [ ]:
model.train()
likelihood.train()

optimizer = torch.optim.Adam(model.parameters(), lr=.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

N_iterations = 500
def train():
    for i in range(N_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward() 
        if i%10 == 0:
            print("Iteration Number: %d" % i)
            print("Loss = %f\n" % loss)
        optimizer.step()

train()

print("Noise: %f " % model.likelihood.noise_covar.noise.item())
print("Length Scale: %f " %  model.covar_module.base_kernel.lengthscale.item())
print("Output Scale: %f " %  model.covar_module.outputscale.item())
print("Mean:  %f " % model.mean_module.constant.item())


Note that the "Length Scale" is the correlation length (in days) learned from the time series. A value of about a day does not bode well for our ability to predict SVDI very far into the future! But let's take a look. It's prediction time.

In [ ]:
Test_Days = N_Train + 30
Start_Test = Start_Day
test_x = np.arange(Start_Test, Start_Test+Test_Days, dtype=np.float64).reshape((Test_Days,1))
test_x = Tensor(test_x)
test_y = output_data_all[Start_Test:Start_Test+Test_Days, long, lat, point[0], point[1]]
test_y = np.array(test_y)
test_y = Tensor(test_y)

# Switch to predictive distribution
model.eval()
likelihood.eval()
    
with torch.no_grad():
    preds = likelihood(model(test_x)) #or model(test_x)

lower, upper = preds.confidence_region() # 2-sigma confidence region

fig1 = plt.figure()
fig1.set_figwidth(8.0)
fig1.set_figheight(8.0)
ax = fig1.add_subplot(1,1,1)

ax.plot(test_x.numpy(),test_y.numpy(), "bo")
ax.plot(test_x.numpy(), preds.mean.numpy(), "r-")
ax.fill_between(test_x.flatten().numpy(), lower.numpy(), upper.numpy(), alpha = 0.5)
ax.axvline(x=Start_Day + N_Train)

plt.show()

The red line is the mean of the predictive gaussian, the blue shaded area is the $2 \sigma$ confidence region produced by the model.

Well, this data sampled at this cadence is a tough modeling assignment. Even in the training region the model is forced to play connect-the-dots, because of the very noisy behavior of the data. In the forecasting region beyond day 120, we can see the effect of the very short ($<1$ day) length scale, which forces the model to its constant prior state in less than a day. Note, however, that as a *probabilistic* forecast the model did quite well, since the $2\sigma$ ($\sim$ 95,5% probability) contour does in fact bracket all the events post-Day 120.

Possible exercises:

1. What happens if you change the parameter for the GP kernel? A Matern kernel with `nu=1.5` instead of `nu=2.5` captures much "rougher" functions.  What do you see?

3. What happens if you change the GP kernel type entirely, e.g. to a Radial Basis Function (RBF) kernel or a periodic kernel? A Matern kernel with $\nu=\infty$ is equivalent to the RBF kernel. 

2. Would you expect to obtain better predictions if you trained on a longer stretch of data? Try it! Explain what you found. (Hint: would the length scale change?)

This was an illustration of the use of GPyTorch, but it did not warm up the computers very much. Let's try harder.

GPs are often used as interpolators. A common and useful application of interpolation in geostatistics is "downscaling".  In downscaling, one interpolates a higher-resolution spatial distribution from a lower one.

Here we will take an $N\times N$ array of tiles, each containing $16\times 16$ points, and downscale the SVDI function on that array. First we will train a GP model to the resulting set of $256 \times N^2$ values. This will bring the [KISS-GP](http://proceedings.mlr.press/v37/wilson15.html) algorithm into play: this is a so-called "inducing point method" that chooses a grid on which to interpolate the data so as to minimize error, while enabling fast Fourier transform methods to speed up the GP training operations that would otherwise cost $\mathcal{O}(N^3)$ in time. The cost now scales as $m\log m$ where $m$ is the grid size (for each dimension).

Following training, we will interpolate the grid by predicting the SVDI values on a $(2N)\times(2N)$ grid of points in the same region, using the standard GP regression technique. This will highlight the [LOVE](https://proceedings.mlr.press/v80/pleiss18a.html) ("Lanczos Variance Estimates") algorithm, which serves to remove the residual $\mathcal{O}(N)$ cost of computation of predictive covariances. It is the combination of KISS-GP and LOVE that allows $\texttt{GPyTorch}$ to claim "constant in time" scaling for GP modeling.

In [ ]:
# Data prep...
#
Num_Tiles = 8 # This is the square of tiles that we will interpolate
Start_Lat = 4; Start_Long = 14
Day_Number = 100

# Extract the input data
tiles_x = input_data_all[Day_Number, Start_Lat:Start_Lat+Num_Tiles, Start_Long:Start_Long+Num_Tiles, :, :, :]

# Move the two latitude axes and the two longitude axes to make them respectively
# adjacent, so that when we reshape the array they go together
tiles_x = np.swapaxes(tiles_x, 1, 2)  # shape: (Num_Tiles, 16, Num_Tiles, 16, 2)
print("tiles_x shape:", tiles_x.shape)

n_train = np.array(tiles_x.shape[:4]).prod() 

train_x = tiles_x.reshape((n_train,-1)) # shape: (Num_Tiles**2 * 256 , 2)
print("train_x shape: ", train_x.shape)
train_x = Tensor(train_x)

# Now the output data
tiles_y = output_data_all[Day_Number, Start_Lat:Start_Lat+Num_Tiles, Start_Long:Start_Long+Num_Tiles, :, :] # shape: (Num_Tiles, 16, Num_Tiles, 16)

# Again, reorder the axes.
tiles_y = np.swapaxes(tiles_y, 1, 2)
print("tiles_y shape", tiles_y.shape)

train_y = tiles_y.reshape(n_train) # shape: (Num_Tiles**2 * 256)
print("train_y shape:", train_y.shape)
train_y = Tensor(train_y) # shape: (Num_Tiles**2 * 256)


Now we set up the model again. Note the lines activating the KISS-GP algorithm.

In [ ]:

class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        self.mean_module = gpytorch.means.ConstantMean()

        # This is the kernel that will be wrapped by KISS-GP
        self.base_covar_module = gpytorch.kernels.MaternKernel(nu = 2.5)

        # The following two parameters are needed by KISS-GP
        num_dims = train_x.shape[-1]
        # A convenience function for computing appropriate grid sizes for this dataset
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x) 

        # This wrapper is only the output_scale
        self.covar_module =gpytorch.kernels.ScaleKernel(
                
                # This wrapper, however, is KISS-GP
                gpytorch.kernels.GridInterpolationKernel(self.base_covar_module, grid_size=grid_size, num_dims=num_dims)
            )
        
     
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)


OK, on to training. Note that we have a much bigger training set now --- big enough that we might not even have attempted it a year or two ago!

In [ ]:

################CUDA######################
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
    likelihood = likelihood.cuda()
    train_x, train_y = train_x.cuda(), train_y.cuda()
######################

optimizer = torch.optim.Adam(model.parameters(), lr=.1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

N_iterations = 81
def train():
    for i in range(N_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward() 
        if i%10 == 0:
            print("Iteration Number: %d" % i)
            print("Loss = %f\n" % loss)
        optimizer.step()
#         scheduler.step(loss)

train()
print("Noise: %f " % model.likelihood.noise_covar.noise.item())
print("Length Scale: %f " %  model.base_covar_module.lengthscale.item())
print("Output Scale: %f " %  model.covar_module.outputscale.item())
print("Mean:  %f " % model.mean_module.constant.item())


The training could have gone on, but life is short.  Let's see how we do with the current model.

We need an output tensor array to hold a $(2N)\times(2N)$ grid of locations that we can interpolate to. This requires some explanation:

[An explanation of the interpolation will be provided in a future version, with a picture, that hopefully helps to justify some of the possibly inscrutable numpy axis gymnastics+broadcasting+arithmetic+reshaping below.  It's geometrically sensible, but staring at the arrays will likely give you a migraine. Sorry about that.]

In [ ]:
# To each original grid point there corresponds, in the new, interpolated grid,
# 4 new points, one each to the NE, SE, SW, and NW. The x and y vector
# components of the displacement from the original point of the new points all
# have magnitude equal to 1/4 of the length of the side of a cell.
#
# That's what the following enigmatic numpy somehow accomplishes.

dlat = tiles_x[0,0,0,0,0] - tiles_x[0,0,1,0,0] # input tile cell increment of latitude
dlong = tiles_x[0,0,0,1,1] - tiles_x[0,0,0,0,1] # input tile cell increment of longitude

test_tiles = np.expand_dims(tiles_x, axis=(2,5))

delta_vecs = np.zeros((1,1,2,1,1,2,2))
delta_vecs[0,0,0,0,0,0,0] = +dlat/4 ; delta_vecs[0,0,0,0,0,0,1] = -dlong/4
delta_vecs[0,0,1,0,0,0,0] = -dlat/4 ; delta_vecs[0,0,1,0,0,0,1] = -dlong/4
delta_vecs[0,0,0,0,0,1,0] = +dlat/4 ; delta_vecs[0,0,0,0,0,1,1] = +dlong/4
delta_vecs[0,0,1,0,0,1,0] = -dlat/4 ; delta_vecs[0,0,1,0,0,1,1] = +dlong/4

test_tiles = test_tiles + delta_vecs

ts = np.array(test_tiles.shape) # shape: (Num_Tiles, 16, 2, Num_Tiles, 16, 2, 2)

test_tiles_shape = [ts[0],ts[1]*ts[2], ts[3],ts[4]*ts[5],ts[6]]
test_tiles = test_tiles.reshape(test_tiles_shape) 
# shape: (Num_Tiles, 32, Num_Tiles, 32, 2)

n_test = np.array(test_tiles.shape[:4]).prod()
test_x = test_tiles.reshape((n_test,-1)) # shape: (Num_Tiles**2 * 1024, 2)
test_x = Tensor(test_x)
if use_cuda:
    test_x = test_x.cuda()


Grid in hand, we can now go into prediction mode. LOVE is activated using a python context as shown below.

In [ ]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.fast_pred_var(): #LOVE is enabled
    preds_kiss = likelihood(model(test_x)) #or model(test_x)
    preds_mean = preds_kiss.mean
    preds_var = preds_kiss.variance

So, how did we do? Let's look at the results, side-by-side with the original:

In [ ]:
odim = 16 * Num_Tiles
ddim = 2 * odim
downscaled_SVDI = preds_mean.cpu().numpy().reshape((ddim,ddim)).T
orig_SVDI = train_y.cpu().numpy().reshape((odim,odim)).T

long0 = tiles_x[0,0,0,0,1] ; long1 = tiles_x[0,0,-1,-1,1]
lat0 = tiles_x [-1,-1,0,0,0] ; lat1 = tiles_x[0,0,0,0,0]

fig = plt.figure()
fig.set_figwidth(17.0)
fig.set_figheight(8.0)

ax1 = fig.add_axes((0.1, 0.1, 0.45, 0.8))
im1 = ax1.imshow(orig_SVDI, cmap=cm.RdYlGn, extent=(long0,long1,lat0,lat1))
ax1.tick_params(axis="both", which="both", labelsize=16)
ax1.set_title("Original SVDI", fontsize=20)

ax2 = fig.add_axes((0.6, 0.1, 0.45, 0.8))
im2 = ax2.imshow(downscaled_SVDI, cmap=cm.RdYlGn, extent=(long0,long1,lat0,lat1))
ax2.tick_params(axis="both", which="both", labelsize=16)
ax2.set_title("Downscaled SVDI", fontsize=20)


plt.show()


The interpolated version definitely looks as if it has higher resolution, so we can claim a success on that score there.

Some possible exercises:

1. Plot the root-variances as colormaps (better put up a colorbar). Can we say anything useful about the interpolation errors?  

2. Does anything interesting happen if you change the kernel, dialing the assumed smoothness of the data up or down? 

3. Try increasing `Num_Tiles`. What do you observe about performance on the GPUs for the training and the predictions phases?  (*In a future version we'll have timers in this thing.*)

A future version of the tutorial will showcase an interesting capability of $\texttt{GPyTorch}$: batch processing of independent GP fits. We will use the same dataset to fit GP models to all the tiles simultaneously, extracting the Length_Scale and Output_Scale parameters, and visualizing them on a map of the US. One such map, produced using $\texttt{R}$ by Tiffany Christian, looks like this:

![SVDI Correlation Lengths](./images/1982_gp.jpeg)

